In [53]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt


df = pd.read_csv('measurements_join_eda.csv', sep=';', decimal=',')

df.head()

,MeasurementDateTime,SensorId,ModuleCode,MeasurementInterval,ModuleName,Number of Records,Unit,Value
0,21.06.2020 19:03,602267e77f522d0007fb01cc,Temperature,00:05:00,SHT3X_Temperature,1,°C,21.5625
1,21.06.2020 19:08,602267e77f522d0007fb01cc,Temperature,00:05:00,SHT3X_Temperature,1,°C,21.5625
2,21.06.2020 19:13,602267e77f522d0007fb01cc,Temperature,00:05:00,SHT3X_Temperature,1,°C,21.5625
3,21.06.2020 19:18,602267e77f522d0007fb01cc,Temperature,00:05:00,SHT3X_Temperature,1,°C,21.5625
4,21.06.2020 19:23,602267e77f522d0007fb01cc,Temperature,00:05:00,SHT3X_Temperature,1,°C,21.5625


In [54]:
# Pętla, która tworzy krótki raport dotyczący odrębnych oraz pustych wartości i dla każdej z kolumn.
for col in df.columns:
    print('Column name:', col)
    print('\t Count of distinct values: ', len(df[col].unique()))
    print('\t Has null: ', df[col].isnull().values.any())
    print()

Column name: MeasurementDateTime
	 Count of distinct values:  46683
	 Has null:  False

Column name: SensorId
	 Count of distinct values:  25
	 Has null:  False

Column name: ModuleCode
	 Count of distinct values:  6
	 Has null:  False

Column name: MeasurementInterval
	 Count of distinct values:  2
	 Has null:  False

Column name: ModuleName
	 Count of distinct values:  6
	 Has null:  False

Column name: Number of Records
	 Count of distinct values:  1
	 Has null:  False

Column name: Unit
	 Count of distinct values:  6
	 Has null:  False

Column name: Value
	 Count of distinct values:  938
	 Has null:  False



In [55]:
# - Tabela nie zawiera żadnych NULL wartości.
# - Kolumny "ModuleCode", "ModuleName", "Unit" zawierają dokładnie tyle samo odrębnych wartości. Bazując na tej informacji oraz obserwacji części tabeli, 
#   możemy założyć, że przedstawiają one tę samą informację. Możemy pozbyć się dwóch z trzech kolumn.
# - Kolumna "Number of Records" nie dostarcza żadnej wartościowej informacji.
# - Kolumny "MeasurementDateTime" oraz "MeasurementInterval" mogą również być uznane, za takie, które przedstawiają tę samą informację.

In [56]:
# Pozbywamy się niepotrzebnych kolumn.
df.drop(['Number of Records', 'ModuleCode', 'ModuleName', 'MeasurementInterval'], inplace=True, axis=1)
df.head()

,MeasurementDateTime,SensorId,Unit,Value
0,21.06.2020 19:03,602267e77f522d0007fb01cc,°C,21.5625
1,21.06.2020 19:08,602267e77f522d0007fb01cc,°C,21.5625
2,21.06.2020 19:13,602267e77f522d0007fb01cc,°C,21.5625
3,21.06.2020 19:18,602267e77f522d0007fb01cc,°C,21.5625
4,21.06.2020 19:23,602267e77f522d0007fb01cc,°C,21.5625


In [57]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 468953 entries, 0 to 468952
Data columns (total 4 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   MeasurementDateTime  468953 non-null  object 
 1   SensorId             468953 non-null  object 
 2   Unit                 468953 non-null  object 
 3   Value                468953 non-null  float64
dtypes: float64(1), object(3)
memory usage: 14.3+ MB


In [58]:
# W tym kroku chcemy zmienić typ danych w kolumnie "MeasurementDateTime" z obiektowych na datowe oraz posortować tabelę, aby zapisy z sensorów zaczynały się od najstarszej daty.
df['MeasurementDateTime'] = pd.to_datetime(df['MeasurementDateTime'], dayfirst=True)
df = df.sort_values(by='MeasurementDateTime')
df.info()
df

<class 'pandas.core.frame.DataFrame'>
Int64Index: 468953 entries, 343551 to 1192
Data columns (total 4 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   MeasurementDateTime  468953 non-null  datetime64[ns]
 1   SensorId             468953 non-null  object        
 2   Unit                 468953 non-null  object        
 3   Value                468953 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 17.9+ MB


,MeasurementDateTime,SensorId,Unit,Value
343551,2020-06-07 12:59:00,602267e87f522d0007fb01e6,Lux,0.000000
340979,2020-06-07 12:59:00,602267e87f522d0007fb01e6,hPa,236.355896
351243,2020-06-07 12:59:00,602267e87f522d0007fb01e6,%,98.669998
351767,2020-06-07 12:59:00,602267e87f522d0007fb01e6,°C,2.500000
289563,2020-06-07 13:00:00,602267e87f522d0007fb01e3,%,33.930000
...,...,...,...,...
1479,2020-09-14 06:11:00,602267e87f522d0007fb01e7,%,38.609997
1191,2020-09-14 06:11:00,602267e87f522d0007fb01e7,Lux,0.000000
1480,2020-09-14 06:16:00,602267e87f522d0007fb01e7,%,39.000000
1768,2020-09-14 06:16:00,602267e87f522d0007fb01e7,°C,18.750000


In [59]:
# W naszej pracy chcemy jedynie przewidzieć zmianę temperatury podczas przewozu.
# W związku z tym interesują nas jedynie recordy, dla których widnieje "°C" w kolumnie "Unit".

df = df.loc[df.Unit == '°C']
df

,MeasurementDateTime,SensorId,Unit,Value
351767,2020-06-07 12:59:00,602267e87f522d0007fb01e6,°C,2.5000
262267,2020-06-07 13:00:00,602267e87f522d0007fb01e2,°C,21.5625
289679,2020-06-07 13:00:00,602267e87f522d0007fb01e3,°C,21.5625
74546,2020-06-07 13:01:00,602267e77f522d0007fb01d3,°C,22.1875
322484,2020-06-07 13:01:00,602267e87f522d0007fb01e4,°C,3.1250
...,...,...,...,...
1764,2020-09-14 05:56:00,602267e87f522d0007fb01e7,°C,18.7500
1765,2020-09-14 06:01:00,602267e87f522d0007fb01e7,°C,18.7500
1766,2020-09-14 06:06:00,602267e87f522d0007fb01e7,°C,18.7500
1767,2020-09-14 06:11:00,602267e87f522d0007fb01e7,°C,18.7500


In [60]:
# Spośród 468953 recordów, 123871 dotyczy zapisów temperatury.
# Sprawdzamy średnią zarejestrowaną temperaturę dla każdego z 25 sensorów.
df_group = df.groupby(['SensorId']).mean()
print(df_group)

                              Value
SensorId                           
602267e77f522d0007fb01ca   4.932607
602267e77f522d0007fb01cb   4.931107
602267e77f522d0007fb01cc  20.935456
602267e77f522d0007fb01cd   4.935001
602267e77f522d0007fb01ce   4.834896
602267e77f522d0007fb01cf   4.132562
602267e77f522d0007fb01d0   3.962662
602267e77f522d0007fb01d1   1.999463
602267e77f522d0007fb01d2  16.007420
602267e77f522d0007fb01d3  20.920984
602267e77f522d0007fb01d5   1.876689
602267e87f522d0007fb01d6  20.091500
602267e87f522d0007fb01d7   2.152099
602267e87f522d0007fb01d8   2.353918
602267e87f522d0007fb01da  20.045310
602267e87f522d0007fb01db  19.908002
602267e87f522d0007fb01dc   2.542276
602267e87f522d0007fb01dd   3.175481
602267e87f522d0007fb01df  -2.406760
602267e87f522d0007fb01e1  -2.208603
602267e87f522d0007fb01e2  20.627365
602267e87f522d0007fb01e3  20.610280
602267e87f522d0007fb01e4  -1.355914
602267e87f522d0007fb01e6  -3.767295
602267e87f522d0007fb01e7  23.161892


C:\Users\aaa\AppData\Local\Temp\ipykernel_6024\2112491485.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_group = df.groupby(['SensorId']).mean()


In [61]:
# Od razu możemy zauważyć, że średnie temperatur wiekszości sensorów waha się od -4 do 5 stopni Celcjusza.
# Jest jednak 9 sensorów, dla których średnia temperatura wynosi ponad 15 stopni Celcjusza.
# Możemy założyć, że dane sensory rejestrowały temperaturę, podczas przejazdy z wyłączoną chłodziarką.

df = df[~df['SensorId'].isin(['602267e77f522d0007fb01cc', '602267e77f522d0007fb01d2', '602267e77f522d0007fb01d3', '602267e87f522d0007fb01d6', '602267e87f522d0007fb01da', '602267e87f522d0007fb01db', '602267e87f522d0007fb01e2', '602267e87f522d0007fb01e3', '602267e87f522d0007fb01e7'])]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77968 entries, 351767 to 468952
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   MeasurementDateTime  77968 non-null  datetime64[ns]
 1   SensorId             77968 non-null  object        
 2   Unit                 77968 non-null  object        
 3   Value                77968 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 3.0+ MB


In [62]:
# Z 123871 zapisów, zostało 77968.
# Posortowanej tabeli (po dacie) nadajemy nowe indexy.
df = df.reset_index()
df

,index,MeasurementDateTime,SensorId,Unit,Value
0,351767,2020-06-07 12:59:00,602267e87f522d0007fb01e6,°C,2.5000
1,322484,2020-06-07 13:01:00,602267e87f522d0007fb01e4,°C,3.1250
2,41676,2020-06-07 13:02:00,602267e77f522d0007fb01d1,°C,1.8750
3,18555,2020-06-07 13:02:00,602267e77f522d0007fb01d0,°C,3.1250
4,107101,2020-06-07 13:03:00,602267e77f522d0007fb01d5,°C,3.4375
...,...,...,...,...,...
77963,468948,2020-08-14 01:44:00,602267e77f522d0007fb01ca,°C,17.5000
77964,468949,2020-08-14 01:49:00,602267e77f522d0007fb01ca,°C,17.5000
77965,468950,2020-08-14 01:54:00,602267e77f522d0007fb01ca,°C,17.5000
77966,468951,2020-08-14 01:59:00,602267e77f522d0007fb01ca,°C,17.1875
